## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [16]:
import pandas as pd
statistic = pd.read_csv('/datasets/data.csv')
statistic.info()
statistic.head(10)


    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Таблица содержит 12 столбцов и 21525 строк. В таблице есть пропущенные значения в столбцах 'days_employed' и 'total_income'.
Тип данных можно изменить с float на int, так как для приведенных значений следует использовать целочисленные значения.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [17]:
print(statistic.isna().sum()) #считаем кол-во отсутствующих значений
print(statistic['income_type'].value_counts())

#замена значений NaN на средний трудовой стаж
days_employed_mean = statistic['days_employed'].mean()
total_income_mean = statistic['total_income'].mean()
statistic['days_employed'] = statistic['days_employed'].fillna(value = days_employed_mean)
statistic['total_income'] = statistic['total_income'].fillna(value = total_income_mean)
statistic.info()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type  

### Вывод

1) Методом isna обнаружено 2174 пропущенных значений в столбцах 'days_employed' и 'total_income'. 
2) Возможно данные не были внесены в таблицу ввиду их отсутствия. Либо пропущенные данные означают отсутствие дохода или рабочего стажа.
3) Проведен анализ столбца 'income_type ' и обнаружено, что только четыре клиента потенциально могут не иметь рабочего стажа и дохода.
На основании этого заменили пропущенные значения средними по столбцу.

### Замена типа данных

In [18]:
statistic['total_income'] = statistic['total_income'].astype('int')
statistic['days_employed'] = statistic['days_employed'].astype('int')
statistic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Для изменения типа данных был применён метод astype, позволяющий изменить тип float на int.

### Обработка дубликатов

In [20]:
statistic['education'] = statistic['education'].str.lower()
print(statistic.duplicated().sum())
statistic.drop_duplicates().reset_index(drop= True)



71


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21451,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21452,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Вывод

1) Для поиска дубликатов используется метод duplicated(), так как он позволяет искать одинаковые строки. Метод value_counts не подходит, так как в столбцах могут встречаться одинаковые значения.
2) Возможно, наличие дубликатов было вызвано повторным внесением одной и той же информации в таблицу.(человеческий фактор)

### Лемматизация

In [38]:

from pymystem3 import Mystem
m = Mystem()

#statistic_purpose = statistic['purpose'].unique()- список использовался для выбора слов классификации целей кредита
# создаём функцию lem_purpose для лемматизации целей кредита
# на основе выбранных слов создано четыре категории целей кредита

def lem_purpose(statistic): 
    lemmas = []
    lemmas = m.lemmatize(statistic)
    for word in lemmas:    
        if word == 'жилье' or word == 'недвижимость': 
            return 'операции с жильем'
        if word == 'свадьба':
            return 'кредит на свадьбу'
        if word == 'автомобиль':
            return 'кредит на автомобиль'
        if word == 'образование':
            return 'кредит на образование'               

    
# создание столбца column_purpose с новыми категориями целей кредита
statistic['column_purpose'] = statistic['purpose'].apply(lem_purpose)
#print(statistic.head())

### Вывод

Была подключена библиотека pymystem3. Она использовалась для лемматизации целей кредита (столбец 'purpose'). Была создана функция lem_purpose, которая добавляет в таблицу statistic новый столбец 'column_purpose' с новыми категориями целей кредита, полученных лемматизацией столбца 'purpose' методом lemmatize().

### Категоризация данных

In [8]:
# категоризация ежемесячного дохода(столбец 'total_income')
# среднее значение 'total_income'
mean_income = statistic['total_income'].sort_values().mean()
#print(mean_income)
#создание словаря с указанием дохода и задолженности
income_total = statistic[['total_income', 'debt']]

# разделяем доходы на две части относительно среднего дохода income_high и income_low
income_high=[]
income_low =[]
for row in income_total['total_income']:
    if row < mean_income:
        income_low += [row]
    else:
        income_high+= [row]

# создаем DataFrame для income_high и находим для него среднее значение
income_high = pd.DataFrame(income_high)
income_high_mean = income_high.mean()
#print(income_high_mean)
# создаем DataFrame для income_low и находим для него среднее значение
income_low = pd.DataFrame(income_low)
income_low_mean = income_low.mean()        
#print(income_low_mean)    

# создаём функцию income_group для категоризации столбца total_income по вычисленным средним значениям
def income_group(word):
    if 0<word<111373 :
        return 'низкий доход'
    if 111373<word<mean_income:
        return 'доход ниже среднего'
    if mean_income<word<236613:
        return 'доход выше среднего'
    if 236613<word:
        return 'высокий доход'
    
income_total['income_group'] = income_total['total_income'].apply(income_group)
print(income_total)

167421.82174216027
      income_type  total_income  debt         income_group
0       сотрудник        253875     0        высокий доход
1       сотрудник        112080     0  доход ниже среднего
2       сотрудник        145885     0  доход ниже среднего
3       сотрудник        267628     0        высокий доход
4       пенсионер        158616     0  доход ниже среднего
...           ...           ...   ...                  ...
21520   компаньон        224791     0  доход выше среднего
21521   пенсионер        155999     0  доход ниже среднего
21522   сотрудник         89672     1         низкий доход
21523   сотрудник        244093     1        высокий доход
21524   сотрудник         82047     0         низкий доход

[21525 rows x 4 columns]


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### Вывод

1) Сделана категоризация по уровню ежемесячного дохода.Уровни дохода были разделены на четыре группы, для упрощения работы с данными.
2) Категоризация по целям кредита была проведена вместе с лемматизацией.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [39]:
statistic_child_dept=statistic[['children','debt']]

statistic_child_dept['children']=statistic_child_dept['children'].replace(-1,1)# заменяю кол-во детей=-1 на 1
statistic_child_dept['children']=statistic_child_dept['children'].replace(20,2)# заменяю кол-во детей=20 на 2

# создание таблицы зависимости задолженности от кол-ва детей
child_debt = statistic_child_dept.groupby('children').agg({'debt': ['sum','count']})
child_debt['conversion'] = child_debt['debt']['sum'] / child_debt['debt']['count']
print(child_debt.sort_values(by=['conversion']))

          debt        conversion
           sum  count           
children                        
5            0      9   0.000000
0         1063  14149   0.075129
3           27    330   0.081818
1          445   4865   0.091470
2          202   2131   0.094791
4            4     41   0.097561


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


### Вывод

Больше кредитов берут люди не имеющие детей. При этом процент должников составляет 7,5%.
Чем больше у клиентов детей, тем меньше они берут кредитов. При этом кол-во клиентов, имеющих от 3 до 5 детей соствляет 2% от всей выборки, что не позволяет сделать однозначный вывод. Для клиентов, имеющих от 0 до 2 детей, задолженность растёт с увеличением кол-ва детей от 7,5% до 9,4%.


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [10]:
statistic_family_debt=statistic[['family_status','debt']]
family_debt = statistic_family_debt.groupby('family_status').agg({'debt': ['sum','count']})
family_debt['conversion'] = family_debt['debt']['sum'] / family_debt['debt']['count']
                           

print(family_debt.sort_values(by=['conversion']))

                      debt        conversion
                       sum  count           
family_status                               
вдовец / вдова          63    960   0.065625
в разводе               85   1195   0.071130
женат / замужем        931  12380   0.075202
гражданский брак       388   4177   0.092890
Не женат / не замужем  274   2813   0.097405


### Вывод

Наибольшее число задолженностей по кредитам у неженатых 9.7% и состоящих в гражданском браке людей 9.2%.
Наименьшее число задолженностей по кредитам у вдовцов 6.5%.
Среднее число задолженностей по кредитам у женатых 7.5% и разведенных людей 7.1%.
Таким образом можно наблюдать зависимость между семейным положением и задолженностью по кредиту.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [11]:

income_debt = income_total.groupby('income_group').agg({'debt': ['sum','count']})
income_debt['conversion'] = income_debt['debt']['sum'] / income_debt['debt']['count']
print(income_debt.sort_values(by=['conversion']))

                    debt       conversion
                     sum count           
income_group                             
высокий доход        226  3248   0.069581
доход выше среднего  512  6385   0.080188
низкий доход         469  5831   0.080432
доход ниже среднего  534  6061   0.088104


### Вывод

Для людей с доходом ниже среднего задолженность по кредиту является наиболее высокой 8.8%
Тогда как, для людей с высоким доходом задолженность по кредиту является наиболее низкой 6.9%
В совокупности, наблюдаем зависимость, что люди с доходом ниже среднего чаще имеют задолженность по кредиту.

- Как разные цели кредита влияют на его возврат в срок?

In [12]:
statistic_income_debt=statistic[['column_purpose','debt']]
purpose_debt = statistic_income_debt.groupby('column_purpose').agg({'debt': ['sum','count']})
purpose_debt['conversion'] = purpose_debt['debt']['sum'] / purpose_debt['debt']['count']
print(purpose_debt.sort_values(by=['conversion']))

                      debt        conversion
                       sum  count           
column_purpose                              
операции с жильем      782  10840   0.072140
кредит на свадьбу      186   2348   0.079216
кредит на образование  370   4022   0.091994
кредит на автомобиль   403   4315   0.093395


### Вывод

Наибольшее число задолженностей по кредитам приходится на 2 категории: кредит на автомобиль 9.3% и на образование 9.2%.

### Шаг 4. Общий вывод

Семейное положение и кол-во детей влияют на факт погашения кредита. Больше должников среди людей не состоящих в каких-либо отношениях. Также клиенты, у которых есть дети чаще имеют задолженности.